## <center> Time series analysis

Importing necessary libraries

In [ ]:
import random
random.seed(42)

import seaborn as sns
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, KFold

import matplotlib.pyplot as plt
%matplotlib inline


def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

We will take real time-series data of total ads watched by hour in a game.

In [ ]:
df = pd.read_csv('../../data/ads_hour.csv',index_col=['Date'], parse_dates=['Date'])
df.head()

In [ ]:
with plt.style.context('bmh'):    
    plt.figure(figsize=(15, 8))
    plt.title('Ads watched (hour ticks)')
    plt.plot(df.ads);

We have everything you could ask for - trend, seasonality, even some outliers.

In this notebook we will concentrate on methods that have proven to be working in practice and can provide quality, comparable to ARIMA models. Namely, feature engineering, selecting and machine learning

- First possible solution - let's get dummies and make 24 new columns out of one. Clear disadvantages - we explode the dimentionality of our data and lose any trace of the cyclical nature of hours.

- Second solution - sine/cosine transformation. To fully understand that approach, read [this short article](https://ianlondon.github.io/blog/encoding-cyclical-features-24hour-time/). But in short - we want to encode hour feature with two new columns, which are sine and cosine transformations of "hour from midnight"

We will use the already familiar `prepareData` function with some modifications - sine/cosine transformation for `hour` and dummy transformation for `weekday` features

In [ ]:
def prepareData(data, lag_start=5, lag_end=14, test_size=0.15):
    """
    series: pd.DataFrame
        dataframe with timeseries

    lag_start: int
        initial step back in time to slice target variable 
        example - lag_start = 1 means that the model 
                  will see yesterday's values to predict today

    lag_end: int
        final step back in time to slice target variable
        example - lag_end = 4 means that the model 
                  will see up to 4 days back in time to predict today

    test_size: float
        size of the test dataset after train/test split as percentage of dataset

    """
    data = pd.DataFrame(data.copy())
    data.columns = ["y"]
    
    # Step 1: calculate test index start position to split data on train test
    test_index = int(len(data) * (1 - test_size))
    
    # Step 2: adding lags of original time series data as features
    for i in range(lag_start, lag_end):
        data["lag_{}".format(i)] = data.y.shift(i)
        
    # Step 3: transforming df index to datetime and creating new variables
    data.index = pd.to_datetime(data.index)
    data["hour"] = data.index.hour
    data["weekday"] = data.index.weekday
        
    # since we will be using only linear models we need to get dummies from weekdays 
    # to avoid imposing weird algebraic rules on day numbers
    data = pd.concat([
        data.drop("weekday", axis=1), 
        pd.get_dummies(data['weekday'], prefix='weekday')
    ], axis=1)
        
    # Step 4: encode hour with sin/cos transformation
    # credits - https://ianlondon.github.io/blog/encoding-cyclical-features-24hour-time/
    data['sin_hour'] = np.sin(2*np.pi*data['hour']/24)
    data['cos_hour'] = np.cos(2*np.pi*data['hour']/24)
    data.drop(["hour"], axis=1, inplace=True)
        

    data = data.dropna()
    data = data.reset_index(drop=True)
    
    
    # Step 5: splitting whole dataset on train and test
    X_train = data.loc[:test_index].drop(["y"], axis=1)
    y_train = data.loc[:test_index]["y"]
    X_test = data.loc[test_index:].drop(["y"], axis=1)
    y_test = data.loc[test_index:]["y"]
    
    return X_train, X_test, y_train, y_test

We'll use the functions on original `df` to prepare datasets necessary for model training. Reserve 30% of data for testing, using initial lag 12 and final lag 48. This way the model will be able to make forecasts twelve steps ahead, having observed data from the previous 1.5 day. 

We'll scale the resulting datasets with the help of `StandardScaler` and create new variables - `X_train_scaled` and `X_test_scaled`.

In [ ]:
X_train, X_test, y_train, y_test = prepareData(df, lag_start=12, lag_end=48, test_size=0.3)
scaler = StandardScaler().fit(X_train)
scaler

In [ ]:
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_train)
X_train_scaled

Now we'll train a simple linear regression on scaled data:

In [ ]:
regr = LinearRegression()
simple = regr.fit(X_train_scaled, X_test_scaled)
simple

We'll check the quality of the model on the training set via cross-validation. To do so we'll create an  object-generator of time series cv folds with the help of  `TimeSeriesSplit`. Set the number of folds to be equal to 5. Then make use of `cross_val_score`, feeding it's `cv` parameter with the created generator. Quality metrics should be `neg_mean_absolute_error`.

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)
a = cross_val_score(regr, X_train_scaled, y_train, cv=tscv, scoring='neg_mean_absolute_error')
b = np.mean(a)
b
c = b * -1
c

**The value of MAE on cross-validation is 4490**

Now let's have a look at the forecast itself. We'll use `plotModelResults` function. 

In [ ]:
def plotModelResults(model, df_train, df_test, y_train, y_test, plot_intervals=False, scale=1.96, cv=tscv):
    """
    Plots modelled vs fact values
    
    model: fitted model 
    
    df_train, df_test: splitted featuresets
    
    y_train, y_test: targets
    
    plot_intervals: bool, if True, plot prediction intervals
    
    scale: float, sets the width of the intervals
    
    cv: cross validation method, needed for intervals
    
    """
    # Step 1: making predictions for test
    prediction = model.predict(df_test)
    
    plt.figure(figsize=(20, 7))
    plt.plot(prediction, "g", label="prediction", linewidth=2.0)
    plt.plot(y_test.values, label="actual", linewidth=2.0)
    
    if plot_intervals:
        # Step 2: calculate cv scores
        cv = cross_val_score(
            model, 
            df_train, 
            y_train, 
            cv=cv, 
            scoring="neg_mean_squared_error"
        )

        # Step 3: calculate cv error deviation
        deviation = np.sqrt(cv.std())
        
        # Step 4: calculate lower and upper intervals
        lower = prediction - (scale * deviation)
        upper = prediction + (scale * deviation)
        
        plt.plot(lower, "r--", label="upper bond / lower bond", alpha=0.5)
        plt.plot(upper, "r--", alpha=0.5)
        
    # Step 5: calculate overall quality on test set
    mae  = mean_absolute_error(prediction, y_test)
    mape = mean_absolute_percentage_error(prediction, y_test)
    plt.title("MAE {}, MAPE {}%".format(round(mae), round(mape, 2)))
    plt.legend(loc="best")
    plt.grid(True);

For model coefficients visualization we'll use the following functions:

In [ ]:
def getCoefficients(model):
    """Returns sorted coefficient values of the model"""
    coefs = pd.DataFrame(model.coef_, X_train.columns)
    coefs.columns = ["coef"]
    coefs["abs"] = coefs.coef.apply(np.abs)
    return coefs.sort_values(by="abs", ascending=False).drop(["abs"], axis=1)    
    

def plotCoefficients(model):
    """Plots sorted coefficient values of the model"""
    coefs = getCoefficients(model)
    
    plt.figure(figsize=(20, 7))
    coefs.coef.plot(kind='bar')
    plt.grid(True, axis='y')
    plt.hlines(y=0, xmin=0, xmax=len(coefs), linestyles='dashed')
    plt.show()

Now, we'll train Lasso regression on cross-validation (`LassoCV`) again feeding the `cv` parameter with the created generator-object. 

In [ ]:
lasso = LassoCV()
simple1 = lasso.fit(X_train_scaled, y_train)
simple1

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)
a1 = cross_val_score(lasso, X_train_scaled, y_train, cv=tscv, scoring='neg_mean_absolute_error')
b1 = np.mean(a1)
b1
c1 = b1 * -1
c1

Perfect, we are still having practically the same model quality, while having less features. Using the function `getCoefficients` we'll find out how many features have been dropped.

In [ ]:
getCoefficients(lasso).head()

Alright, we have some features dropped. But what if we want to go further and transform our linear-dependant features into more compact representation? To do so we'll use principal component analysis.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline

def plotPCA(pca):
    """
    Plots accumulated percentage of explained variance by component
    
    pca: fitted PCA object
    """
    components = range(1, pca.n_components_ + 1)
    variance = np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)
    plt.figure(figsize=(20, 10))
    plt.bar(components, variance)
    
    # additionally mark the level of 95% of explained variance 
    plt.hlines(y = 95, xmin=0, xmax=len(components), linestyles='dashed', colors='red')
    
    plt.xlabel('PCA components')
    plt.ylabel('variance')
    plt.xticks(components)
    plt.show()

In [ ]:
# Step 1: Create PCA object: pca
pca = PCA()

# Step 2: Train PCA on scaled data
simple2 = pca.fit(X_train_scaled, y_train)
simple2

# Step 3: plot explained variance
plotPCA(simple2)

**We can observe that the minimum number of components needed to explain at least 95% of variance of the train dataset, is 9 components.**

We'll create the `pca` object once again, this time setting inside it the optimal number of components (explaining at least 95% of variance). After that we'll create two new variables - `pca_features_train` and `pca_features_test`, assigning to them pca-transformed scaled datasets.

In [ ]:
# Create PCA object: pca
pca = PCA(n_components=9)

pca_features_train = pca.fit_transform(X_train_scaled) 
pca_features_test = pca.fit_transform(X_test_scaled)

Now we'll train linear regression on pca features and plot its forecast.

In [ ]:
regr = LinearRegression()
simple3 = regr.fit(pca_features_train, y_train)
simple3

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)
a2 = cross_val_score(regr, pca_features_train, y_train, cv=tscv, scoring='neg_mean_absolute_error')
b2 = np.mean(a2)
b2
c2 = b2 * -1
c2

**Now with the model trained on pca-transformed features, the MAE of linear regression is 4663**